In [481]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
#pd.set_option('display.float_format', '{:.2f}'.format)
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [482]:
test = pd.read_csv('test_listo.csv')
train = pd.read_csv('train_listo.csv')

In [483]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16883 entries, 0 to 16882
Data columns (total 58 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Opportunity_ID                      16883 non-null  int64  
 1   Cantidad de Items Asociados         16883 non-null  int64  
 2   ID                                  16883 non-null  int64  
 3   Region                              16883 non-null  object 
 4   Territory                           16883 non-null  object 
 5   Pricing, Delivery_Terms_Quote_Appr  16883 non-null  int64  
 6   Pricing, Delivery_Terms_Approved    16883 non-null  int64  
 7   Bureaucratic_Code_0_Approval        16883 non-null  int64  
 8   Bureaucratic_Code_0_Approved        16883 non-null  int64  
 9   Submitted_for_Approval              16883 non-null  int64  
 10  Bureaucratic_Code                   16883 non-null  object 
 11  Account_Created_Date                16883

In [484]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Opportunity_ID                      2551 non-null   int64  
 1   Cantidad de Items Asociados         2551 non-null   int64  
 2   ID                                  2551 non-null   int64  
 3   Region                              2551 non-null   object 
 4   Territory                           2551 non-null   object 
 5   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 6   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 7   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 8   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 9   Submitted_for_Approval              2551 non-null   int64  
 10  Bureaucratic_Code                   2551 non-null   object 
 11  Account_Created_Date                2551 no

In [485]:
train.columns

Index(['Opportunity_ID', 'Cantidad de Items Asociados', 'ID', 'Region',
       'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date

In [486]:
len(set(train.Opportunity_Name))

9791

In [487]:
len(set(test.Opportunity_Name))

1567

In [488]:
train.Stage.value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

In [489]:
train.loc[train['Stage']=='Closed Won','Stage']=1
train.loc[train['Stage']=='Closed Lost','Stage']=0

In [490]:
#train.fillna(0,inplace=True)
#test.fillna(0,inplace=True)

In [491]:
#train.Source.fillna('nada',inplace=True)
#test.Source.fillna('nada',inplace=True)

* One Hot Encoding y Label Encoding

In [492]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

In [493]:
def LabelEncord_categorical(df):
    categorical_features = ['Region','Territory','Bureaucratic_Code','Source ','Billing_Country', 'Account_Name','Account_Owner',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family', 'Product_Name','ASP_Currency','Prod_Category_A', 'Hemisferio', 'Estación']
    for features in categorical_features:
        le = LabelEncoder()
        df[features] = le.fit_transform(df[features].astype(str))
    return df

def dummies(df):
    categorical_features = ["year","provincia","tipodepropiedad",'ciudad']
    for features in categorical_features:
        dummies =  pd.get_dummies(df[features])
        df = pd.concat([df, dummies],axis=1)
    return df

def pre_processing(df):
    df = LabelEncord_categorical(df)
    return df

#df = pre_processing(df)
train = pre_processing(train)
test = pre_processing(test)

In [494]:
train.Delivery_Quarter.value_counts()

1    4423
0    4295
3    4134
2    4031
Name: Delivery_Quarter, dtype: int64

In [495]:
train.head(10)

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,0,1,27761,2,46,1,1,1,1,0,...,5272800.0,EUR,5272800.0,0,0,2,4,5272800.0,5.963537e+06,5.963537e+06
1,1,1,27760,2,46,0,0,0,0,0,...,48230.0,EUR,48230.0,1,0,2,4,48230.0,5.454813e+04,5.454813e+04
2,2,1,27446,0,43,0,0,0,0,0,...,83865.6,USD,83865.6,1,0,0,1,83865.6,8.386560e+04,8.386560e+04
3,3,1,16808,0,43,1,0,1,0,0,...,7421881.5,USD,7421881.5,0,0,0,1,7421881.5,7.421882e+06,7.421882e+06
4,4,1,16805,0,43,1,0,1,0,0,...,13357192.5,USD,13357192.5,0,0,0,1,13357192.5,1.335719e+07,1.335719e+07
5,5,1,16802,0,43,1,0,1,0,0,...,14838277.5,USD,14838277.5,0,0,0,1,14838277.5,1.483828e+07,1.483828e+07
6,6,1,16799,0,43,1,0,1,0,0,...,2659494.6,USD,2659494.6,0,0,0,1,2659494.6,2.659495e+06,2.659495e+06
7,7,1,27455,0,43,1,1,1,1,0,...,50688.0,USD,50688.0,1,0,0,1,50688.0,5.068800e+04,5.068800e+04
8,8,5,24353,1,31,1,0,0,0,0,...,15600.0,JPY,470400.0,0,0,0,1,470400.0,4.228896e+03,8.457792e+02
9,8,5,24355,1,31,1,0,0,0,0,...,4400.0,JPY,470400.0,0,0,0,1,470400.0,4.228896e+03,8.457792e+02


In [496]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,10689,3,6140,2,15,1,1,1,1,0,...,EUR,162240.0,EUR,367419.0,0,2,2,367419.0,415550.8890,138516.9630
1,10689,3,6146,2,15,1,1,1,1,0,...,EUR,78624.0,EUR,367419.0,0,2,2,367419.0,415550.8890,138516.9630
2,10689,3,6151,2,15,1,1,1,1,0,...,EUR,126555.0,EUR,367419.0,0,2,2,367419.0,415550.8890,138516.9630
3,10690,5,6118,2,15,1,1,1,1,0,...,EUR,243360.0,EUR,757783.5,0,2,2,757783.5,857053.1385,171410.6277
4,10690,5,6124,2,15,1,1,1,1,0,...,EUR,157248.0,EUR,757783.5,0,2,2,757783.5,857053.1385,171410.6277


In [497]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)
train.rename(columns={'Estación':'Estacion'},inplace=True)
test.rename(columns={'Estación':'Estacion'},inplace=True)

In [498]:
categorical_features = ['Region','Territory','Bureaucratic_Code','Source ','Billing_Country', 'Account_Name','Account_Owner',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family', 'Product_Name','ASP_Currency','Prod_Category_A', 'Hemisferio', 'Estacion']

In [499]:
# Aca saco las columnas(labels que no quiero que las tome mi algoritmo)
features = list(set(train.columns) - set(['ID','Stage','Sales_Contract_No','Opportunity_ID','Opportunity_Created_Date','Account_Created_Date','Opportunity_Name',\
                                        'Price', 'Currency', 'ASP', 'ASP_(converted)_Currency', 'ASP_(converted)',\
                                        'Planned_Delivery_Start_Date','Brand','Size', 'Planned_Delivery_End_Date', 'Month',\
                                         'Delivery_Year', 'Actual_Delivery_Date','Product_Type','Product_Category_B',\
                                        'Total_Amount_Currency', 'Total_Amount','Last_Activity',\
                                        'Quote_Expiry_Date', 'Last_Modified_Date',\
                                        'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount','Account_Created_Date',]))
features

['Delivery_Terms',
 'Bureaucratic_Code_0_Approval',
 'Product_Name',
 'Opportunity_Owner',
 'precio_por_items',
 'Product_Family',
 'Hemisferio',
 'Source ',
 'Region',
 'Pricing, Delivery_Terms_Approved',
 'Quote_Type',
 'Account_Name',
 'Prod_Category_A',
 'Billing_Country',
 'Territory',
 'Bureaucratic_Code',
 'Opportunity_Type',
 'Total_moneda_local',
 'ASP_Currency',
 'total_en_dolares',
 'Delivery_Quarter',
 'Pricing, Delivery_Terms_Quote_Appr',
 'Estacion',
 'Account_Type',
 'Last_Modified_By',
 'Bureaucratic_Code_0_Approved',
 'Submitted_for_Approval',
 'Account_Owner',
 'Cantidad de Items Asociados',
 'TRF']

In [500]:
train = train.drop_duplicates(subset='Opportunity_ID',keep='first') 
test = test.drop_duplicates(subset='Opportunity_ID',keep='first') 

In [501]:
X = train[features]
Y = train['Stage']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [502]:
x_train

,Delivery_Terms,Bureaucratic_Code_0_Approval,Product_Name,Opportunity_Owner,precio_por_items,Product_Family,Hemisferio,Source,Region,"Pricing, Delivery_Terms_Approved",...,Delivery_Quarter,"Pricing, Delivery_Terms_Quote_Appr",Estacion,Account_Type,Last_Modified_By,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Account_Owner,Cantidad de Items Asociados,TRF
15295,4,1,133,52,4.332600e+05,36,0,4,0,1,...,3,1,1,0,52,1,0,46,3,1
4819,4,1,218,16,5.107190e+05,67,0,0,0,1,...,0,1,1,2,15,1,0,14,2,1
6066,4,0,144,37,0.000000e+00,35,0,0,1,0,...,2,1,2,2,35,0,0,32,1,0
13659,4,1,144,37,2.711384e+04,35,0,0,1,0,...,0,1,2,2,35,0,0,32,1,0
8771,2,0,38,4,1.740609e+05,3,0,12,2,0,...,3,0,3,4,31,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8753,2,0,64,4,4.869803e+04,15,0,13,2,0,...,2,0,3,0,31,0,0,2,2,0
5920,4,1,337,16,9.360435e+05,111,0,12,0,1,...,2,1,0,2,31,1,0,14,1,3
5919,2,1,133,6,4.017878e+05,36,0,12,2,0,...,2,1,0,4,5,0,0,4,1,1
16760,4,1,204,11,1.626267e+07,61,0,12,0,0,...,1,1,1,4,44,0,0,14,1,25


In [503]:
y_train

15295    1
4819     0
6066     0
13659    1
8771     0
        ..
8753     1
5920     0
5919     0
16760    0
11767    0
Name: Stage, Length: 7832, dtype: int64

In [504]:
y_train.value_counts()

1    4028
0    3804
Name: Stage, dtype: int64

In [507]:
lgb_train = lgb.Dataset(x_train, y_train,feature_name = features, categorical_feature = categorical_features)

In [508]:
num_of_classes=len(train.Stage.unique())

In [509]:
params = {
'task': 'train'
, 'boosting_type': 'gbdt'
, 'objective': 'multiclass'
, 'num_class': num_of_classes
, 'metric': 'multi_logloss'
, 'min_data': 1
, 'verbose': -1
}

In [510]:
gbm = lgb.train(params, lgb_train, num_boost_round=50)

C:\Users\mikef\Anaconda3\envs\Ambiente\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [480]:
#### Formato de subida del archivo
scores = cross_val_score(xgb1, train[features], train['Stage'], cv=3)
scores

array([0.765625  , 0.75153186, 0.71467974])

In [511]:
pred = gbm.predict(x_test[features])[:,1]

In [512]:
#list(preds)

In [513]:
len(pred)

1959

In [514]:
len(x_test)

1959

In [515]:
x_test['target']=pred

C:\Users\mikef\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [516]:
x_test['target']

13096    0.897334
622      0.972724
4549     0.112663
12825    0.394926
6493     0.938915
           ...   
11751    0.159231
16468    0.887454
4521     0.860171
7234     0.924143
10465    0.387261
Name: target, Length: 1959, dtype: float64

In [517]:
pred = gbm.predict(test[features])[:,1]

In [518]:
len(pred)

1567

In [519]:
len(test['Opportunity_ID'])

1567

In [520]:
test['target']=pred

In [521]:
test['target'].value_counts()

0.342132    10
0.321645    10
0.198606     7
0.199273     6
0.201252     6
            ..
0.117091     1
0.164143     1
0.067603     1
0.124081     1
0.174294     1
Name: target, Length: 1381, dtype: int64

In [415]:
test[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.126165
3,10690,0.108133
8,10691,0.650849
9,10692,0.037898
15,10693,0.146673
...,...,...
2545,12364,0.185192
2547,12365,0.328023
2548,12366,0.263170
2549,12367,0.726887


In [523]:
test[['Opportunity_ID', 'target']].to_csv("subidas/lgbm_label_enc_v1.csv", index=False)